In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json

In [ ]:
with open('drive/MyDrive/nlp_project/tags_nurkz.json', 'r', encoding="utf-8") as f:
  tags = json.load(f)
tags

['үй',
 'кітап',
 'сыртқы саясат',
 'мереке',
 'заңбұзушылық',
 'экономика',
 'зейнетақы мен өтемақы',
 'төтенше жағдай',
 'жазатайым оқиға',
 'өзге спорт түрлері',
 'ойын-сауық',
 'денсаулық мәселесі',
 'коронавирус',
 'мансап',
 'астрология',
 'қоғам',
 'әлемде',
 'қазақстан',
 'мектеп',
 'жеке қаржы',
 'қор нарығы',
 'спорт аясындағы жаңалықтар',
 'жеке өмір',
 'банктер',
 'психология',
 'күрес',
 'ана бақыты',
 'сұлулық',
 'түс жору',
 'футбол',
 'хоккей',
 'айыппұл мен салық',
 'рецепт',
 'біз қалай жұмыс істейміз',
 'денсаулық сақтау',
 'салауатты өмір салты',
 'мма',
 'қызық ақпарат',
 'ханшайым',
 'бокс',
 'дұрыс тамақтану',
 'оқыс оқиға',
 'олимпиада',
 'тағайындау',
 'жанжал',
 'жол апаты',
 'имидж',
 'калейдоскоп',
 'сақтандыру']

In [ ]:
len(tags)

49

In [ ]:
unique_tags = set(tag for tag in tags)
print(len(unique_tags))

49


In [ ]:
!ls 'drive/MyDrive/nlp_project/'

links_nurkz.json  nlp_project.ipynb  nurkz_df.csv  tags_nurkz.json


In [ ]:
nurkz_df = pd.read_csv('drive/MyDrive/nlp_project/nurkz_df.csv', dtype=str)

In [ ]:
nurkz_df.head()

,text,tags,url
0,Бие сүтін адам өмірін ұзартатын сусын ретінд...,салауатты өмір салты,https://kaz.nur.kz/health/healthy-lifestyle/22...
1,Қыстың аязды күндерінде көлікті дұрыс қыздыр...,салауатты өмір салты,https://kaz.nur.kz/health/healthy-lifestyle/22...
2,АҚШ-тың Азық-түлік және дәрі-дәрмек сапасын ...,салауатты өмір салты,https://kaz.nur.kz/health/healthy-lifestyle/21...
3,Е дәрумені терінің саулығы мен мидың қызметі...,салауатты өмір салты,https://kaz.nur.kz/health/healthy-lifestyle/21...
4,Сынаптан жасалған дене қызуын өлшейтін құрал...,салауатты өмір салты,https://kaz.nur.kz/health/healthy-lifestyle/21...


In [ ]:
nurkz_df.shape

(15624, 3)

In [ ]:
nurkz_df.drop_duplicates().shape

(15624, 3)

## data prep

In [ ]:
nurkz_df.isna().sum()

,0
text,0
tags,0
url,0
tag_in_list,0


In [ ]:
len(nurkz_df['tags'].unique())

44

In [ ]:
def to_list(row):
  word = [row]
  return word

In [ ]:
nurkz_df['tag_in_list'] = nurkz_df['tags'].apply(to_list)

In [ ]:
nurkz_df['tag_in_list'][0]

['салауатты өмір салты']

In [ ]:
tags_list = nurkz_df['tag_in_list'].tolist()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform(tags_list)

In [ ]:
mlb.classes_

array(['айыппұл мен салық', 'ана бақыты', 'астрология', 'банктер', 'бокс',
       'денсаулық мәселесі', 'денсаулық сақтау', 'дұрыс тамақтану',
       'жазатайым оқиға', 'жанжал', 'жеке қаржы', 'жеке өмір',
       'жол апаты', 'заңбұзушылық', 'зейнетақы мен өтемақы', 'имидж',
       'коронавирус', 'кітап', 'күрес', 'мансап', 'мектеп', 'мереке',
       'мма', 'ойын-сауық', 'олимпиада', 'оқыс оқиға', 'психология',
       'рецепт', 'салауатты өмір салты', 'сақтандыру',
       'спорт аясындағы жаңалықтар', 'сыртқы саясат', 'сұлулық',
       'тағайындау', 'түс жору', 'төтенше жағдай', 'футбол', 'хоккей',
       'экономика', 'қазақстан', 'қор нарығы', 'қызық ақпарат', 'үй',
       'өзге спорт түрлері'], dtype=object)

In [ ]:
for i in nurkz_df['tags'].unique():
  if i not in mlb.classes_:
    print(i)

In [ ]:
import numpy as np

binary_labels.shape

(14355, 44)

In [ ]:
texts = nurkz_df['text'].tolist()

In [ ]:
texts[0]

'  Бие сүтін адам өмірін ұзартатын сусын ретінде қарастырар болсақ, соның ішінде саумалдың емдік қасиеті зор. Осы сусын арқылы қырыққа жуық аурудың алдын алуға болады екен – рак клеткаларына тосқауыл болатыны да айтылады. Соңғы кезде саумалмен емдеу де ел арасында кеңінен тарап келеді. Саумал туралы толығырақ мәліметті мақаладан оқи аласыз. Саумал – жаңа сауылған бие сүті. Дәмі тәтті, көпіршіп тұратын, жұмсақ әрі тез сіңетін қасиеті бар биологиялық табиғи өнім.  Саумал туралы  мақаладажаңа сауылған бие сүті физикалық және химиялық құрамы жағынан ана сүтіне өте жақын болып саналатыны, бұл тұрғыдан алғанда ешкі сүтінен де жақын екені айтылған.  Уикипедиядакелтірілген мәліметке сүйенсек, бие сүті құрамына қарай ана сүтіне өте жақын, альбумин тобына жатады. 500 мл бие сүті ағзаға қажетті С дәруменінің тәуліктік мөлшерін қамтамасыз етеді.  Саумалдың құрамында белок, амин қышқылы, тез еритін май, оттегі және организмге керек кальций, натрий, фосфор, калий, темір, магний, бром сияқты минералд

## Model

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, binary_labels, test_size=0.2, random_state=42)

In [ ]:
from torch.utils.data import Dataset
import torch
from transformers import AutoTokenizer

MODEL_NAME = "kz-transformers/kaz-roberta-conversational"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class NewsDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_texts, train_labels)
val_dataset = NewsDataset(val_texts, val_labels)


config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.69M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(mlb.classes_),
    problem_type="multi_label_classification"
)
model = model.to(device)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=25,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=1,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()


model.safetensors:   0%|          | 0.00/334M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at kz-transformers/kaz-roberta-conversational and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,0.119300,0.116210
2,0.107200,0.106202
3,0.089200,0.087423
4,0.068400,0.065771
5,0.053300,0.051879
6,0.040900,0.043919
7,0.034200,0.040098
8,0.029400,0.036439
9,0.023000,0.034386
10,0.019800,0.033434


TrainOutput(global_step=4500, training_loss=0.03475045388854212, metrics={'train_runtime': 3969.0189, 'train_samples_per_second': 72.335, 'train_steps_per_second': 1.134, 'total_flos': 3.80598760599552e+16, 'train_loss': 0.03475045388854212, 'epoch': 25.0})

In [ ]:
import joblib

model.save_pretrained("nurkz-bert")
tokenizer.save_pretrained("nurkz-bert")
joblib.dump(mlb, "label_binarizer.pkl")


['label_binarizer.pkl']

In [ ]:
# Предсказание на обучающей выборке
train_preds = trainer.predict(train_dataset)
y_train_pred = (train_preds.predictions > 0.2).astype(int)
y_train_true = train_labels  # они у тебя уже есть

# Метрики на train
from sklearn.metrics import classification_report
print("📊 Метрики на TRAIN:")
print(classification_report(y_train_true, y_train_pred, target_names=mlb.classes_))


📊 Метрики на TRAIN:
                            precision    recall  f1-score   support

         айыппұл мен салық       1.00      1.00      1.00       322
                ана бақыты       0.99      1.00      1.00       305
                астрология       1.00      1.00      1.00       311
                   банктер       1.00      1.00      1.00       327
                      бокс       1.00      1.00      1.00       330
        денсаулық мәселесі       1.00      0.99      1.00       319
          денсаулық сақтау       0.00      0.00      0.00        20
           дұрыс тамақтану       1.00      1.00      1.00       134
           жазатайым оқиға       1.00      1.00      1.00       333
                    жанжал       1.00      0.99      1.00       315
                жеке қаржы       1.00      1.00      1.00       326
                 жеке өмір       1.00      1.00      1.00       313
                 жол апаты       1.00      1.00      1.00       307
              заңбұзушылық 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.metrics import classification_report

# Предсказание
preds = trainer.predict(val_dataset)
y_pred = (preds.predictions > 0.2).astype(int)
y_true = val_labels

# Отчет
print(classification_report(y_true, y_pred, target_names=mlb.classes_))


                            precision    recall  f1-score   support

         айыппұл мен салық       0.84      0.79      0.82        78
                ана бақыты       0.82      0.78      0.80        81
                астрология       0.99      0.94      0.97        89
                   банктер       0.82      0.77      0.79        73
                      бокс       0.86      0.90      0.88        70
        денсаулық мәселесі       0.81      0.63      0.71        81
          денсаулық сақтау       0.00      0.00      0.00         4
           дұрыс тамақтану       0.91      0.71      0.79        41
           жазатайым оқиға       0.76      0.81      0.78        67
                    жанжал       0.76      0.65      0.70        83
                жеке қаржы       0.77      0.65      0.71        74
                 жеке өмір       0.63      0.59      0.61        87
                 жол апаты       0.90      0.89      0.90        93
              заңбұзушылық       0.79      0.69

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Save model param

In [ ]:
import shutil

shutil.make_archive('nurkz-bert', 'zip', 'nurkz-bert')


'/content/nurkz-bert.zip'

In [ ]:
from google.colab import files

# Download the zipped model
files.download('nurkz-bert.zip')

# Download the label binarizer
files.download('label_binarizer.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil

shutil.make_archive('logs', 'zip', 'logs')

# Download the zipped model
files.download('logs.zip')


shutil.make_archive('results', 'zip', 'results')

# Download the zipped model
files.download('results.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>